## 12190247 Mid term Colab related file

In [1]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *

     |████████████████████████████████| 9.7 MB 10.1 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to un

### Data File

In [2]:
Demand = {
   'Inch':   125,        # Incheon
   'Sel':   175,        # Seoul
   'Suw':   225,        # Suwon
   'Ams':   250,        # Asan
   'Utr':   225,        # Ulsan
   'Hag':   200         # Pohang
}

Supply = {
   'Busn':   600,        # Busan
   'Mok':   650         # Mokpo
}

T = {
    ('Inch','Busn'): 1000,
    ('Inch','Mok'): 2.5,
    ('Sel','Busn'): 2.5,
    ('Sel','Mok'): 1000,
    ('Suw','Busn'): 1.6,
    ('Suw','Mok'): 2.0,
    ('Ams','Busn'): 1.4,
    ('Ams','Mok'): 1.0,
    ('Utr','Busn'): 0.8,
    ('Utr','Mok'): 1.0,
    ('Hag','Busn'): 1.4,
    ('Hag','Mok'): 0.8
}

In [3]:
# Step 0: Create an instance of the model
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

# Step 1: Define index sets
CUS = list(Demand.keys())
SRC = list(Supply.keys())

# Step 2: Define the decision 
model.x = Var(CUS, SRC, domain = NonNegativeReals)

# Step 3: Define Objective
model.Cost = Objective(
    expr = sum([T[c,s]*model.x[c,s] for c in CUS for s in SRC]),
    sense = minimize)

# Step 4: Constraints
model.src = ConstraintList()
for s in SRC:
    model.src.add(sum([model.x[c,s] for c in CUS]) <= Supply[s])
        
model.dmd = ConstraintList()
for c in CUS:
    model.dmd.add(sum([model.x[c,s] for s in SRC]) == Demand[c])
    
results = SolverFactory('cbc').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1705.0
  Upper bound: 1705.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 13
  Number of nonzeros: 6
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created 

In [4]:
for c in CUS:
    for s in SRC:
        print(c, s, model.x[c,s]())

Inch Busn 0.0
Inch Mok 125.0
Sel Busn 175.0
Sel Mok 0.0
Suw Busn 225.0
Suw Mok 0.0
Ams Busn 0.0
Ams Mok 250.0
Utr Busn 200.0
Utr Mok 25.0
Hag Busn 0.0
Hag Mok 200.0


In [5]:
if 'ok' == str(results.Solver.status):
    print("Total Shipping Costs = ",model.Cost())
    print("\nShipping Table:")
    for s in SRC:
        for c in CUS:
            if model.x[c,s]() > 0:
                print("Ship from ", s," to ", c, ":", model.x[c,s]())
else:
    print("No Valid Solution Found")

Total Shipping Costs =  1705.0

Shipping Table:
Ship from  Busn  to  Sel : 175.0
Ship from  Busn  to  Suw : 225.0
Ship from  Busn  to  Utr : 200.0
Ship from  Mok  to  Inch : 125.0
Ship from  Mok  to  Ams : 250.0
Ship from  Mok  to  Utr : 25.0
Ship from  Mok  to  Hag : 200.0


In [6]:
if 'ok' == str(results.Solver.status):
    print("\nSources:")
    print("Source      Capacity   Shipped    Margin")
    for m in model.src.keys():
        s = SRC[m-1]
        print("{0:10s}{1:10.1f}{2:10.1f}{3:10.4f}".format(s,Supply[s],model.src[m](),model.dual[model.src[m]]))
else:
    print("No Valid Solution Found")


Sources:
Source      Capacity   Shipped    Margin
Busn           600.0     600.0   -0.2000
Mok            650.0     600.0    0.0000


In [7]:
if 'ok' == str(results.Solver.status):    
    print("\nCustomers:")
    print("Customer      Demand   Shipped    Margin")
    for n in model.dmd.keys():
        c = CUS[n-1]
        print("{0:10s}{1:10.1f}{2:10.1f}{3:10.4f}".format(c,Demand[c],model.dmd[n](),model.dual[model.dmd[n]]))
else:
    print("No Valid Solution Found")


Customers:
Customer      Demand   Shipped    Margin
Inch           125.0     125.0    2.5000
Sel            175.0     175.0    2.7000
Suw            225.0     225.0    1.8000
Ams            250.0     250.0    1.0000
Utr            225.0     225.0    1.0000
Hag            200.0     200.0    0.8000
